In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

# plot all PCC
# bandingin data ctilde=1, ctilde=1e7, dan ctilde=0 (_0, _1, dan _2)

radmass_0=np.loadtxt("radmassSL_ct=1.dat")[:, :]
dataEden_0=radmass_0[:,1]
dataRad_0=radmass_0[:,2]
dataMass_0=radmass_0[:,3]
dataCompactness_0=radmass_0[:,4]

radmass_1=np.loadtxt("radmassSL_ct=1e7.dat")[:, :]
dataEden_1=radmass_1[:,1]
dataRad_1=radmass_1[:,2]
dataMass_1=radmass_1[:,3]
dataCompactness_1=radmass_1[:,4]

radmass_0=np.loadtxt("radmassSL_ct=0.dat")[:, :]
dataEden_0=radmass_0[:,1]
dataRad_0=radmass_0[:,2]
dataMass_0=radmass_0[:,3]
dataCompactness_0=radmass_0[:,4]

fig, ax = plt.subplots(3, 1, figsize = (6,15))

# ax[0].plot(dataRad_0, dataMass_0, label="ct=1")
ax[0].plot(dataRad_1, dataMass_1, label="ct=1e7")
ax[0].plot(dataRad_1, dataMass_1, label="ct=0")
ax[0].set_xlabel(r'$R$ (km)', fontsize=16)
ax[0].set_ylabel(r'$M$ ($M_{\odot}$)', fontsize=16)

# ax[1].plot(dataEden_0, dataMass_0, label="ct=1")
ax[1].plot(dataEden_1, dataMass_1, label="ct=1e7")
ax[1].plot(dataEden_1, dataMass_1, label="ct=0")
ax[1].set_xlabel(r'$\epsilon_c$ ($10^3$ MeV fm$^{-3}$)', fontsize=16)
ax[1].set_ylabel(r'$M$ ($M_{\odot}$)', fontsize=16)

# ax[2].plot(dataCompactness_0, dataMass_0, label="ct=1")
ax[2].plot(dataCompactness_1, dataMass_1, label="ct=1e7")
ax[2].plot(dataCompactness_1, dataMass_1, label="ct=0")
ax[2].set_xlabel(r'$C=2GM/R$', fontsize=16)
ax[2].set_ylabel(r'$M$ ($M_{\odot}$)', fontsize=16)

ax[0].legend()

plt.show()

In [ ]:
# kita hitung tau echo dari hasil Fortran, karena hasil Python masih kacau!

import array
from scipy.integrate import trapz

import cmath #To help us out with the complex square root
import numpy as np #For the arrays
import matplotlib.pyplot as plt #Visualization

# deleting profile and radmass files
# os.remove("radmass.txt")
# os.remove("profile.txt")

# some constants
GS = 1.325 * 10**(-12) # Newton constant in fm^3 / MeV m^2
MSS = 1.1155 * 10**(15) # Sun's mass in MeV m^3 / fm^3
PI=np.pi

profile=np.loadtxt("profil_CT=.100D+01_aniso.dat")[:, :]
profr0=profile[:,0]/1000
profb0=profile[:,1]
profP0=profile[:,2]
profm0=profile[:,3]
profL0=profile[:,4]/1000000
proff0=profile[:,5]

maxdata=len(profr0)-1
Rfs=3*GS*MSS*profm0[maxdata]
R=profr0[maxdata]*1000
M=profm0[maxdata]
print("R=",R,", tapi 3GM=",Rfs)

# tau echo rumus approx yg integrasi dari permukaan ke photon sphere saja
def tauapp(r,m):
    return 4*GS*MSS*m*abs(np.log(r/(2*GS*MSS*m)-1)) 
compactness=GS*MSS*M/R
tauapprox= tauapp(R,M)
print("C=",compactness)
print("tauapprox=",tauapprox)

# hitung tau tanpa approx

def rtortoise(r,m):  #definisi r tortoise
    return r + 2*GS*MSS*m*np.log( r/(2*GS*MSS*m)-1 )

# hitung dari permukaan ke photon sphere
tambahan = 2*( rtortoise(Rfs,M) - rtortoise(R,M) ) 
print("tau_int =",tambahan)

# hitung dari pusat ke permukaan
function=1/np.sqrt(profb0*proff0)
h=1000*(profr0[1]-profr0[0])
I_trap = (h/2)*(function[0] + 2 * sum(function[1:maxdata]) + function[maxdata])
# I_trapz = trapz(function,1000*profr0)
print("tau_ext =",I_trap)
# print("tau_ext_auto =",I_trapz)

# tambahkan hasil interior + eksterior
tau=I_trap+tambahan
print("tau =",tau)

# frekuensi
freq_approx=PI/tauapprox*3.e+8
freq_direct=PI/tau*3.e+8
print("freq_approx =",freq_approx, ", freq_direct =",freq_direct, ", M=",M)